In [1]:
import json
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

Read in Data Frame

In [2]:
df = pd.read_csv('sqr_no_comments.csv')

Create a list of all DBN's

In [3]:
dbn_list = list(df.dbn)

In [4]:
len(dbn_list)

1098

In [5]:
dbn_list = dbn_list[1000:]

Get the website

In [6]:
# Create an instance of ChromeOptions
options = webdriver.ChromeOptions()
# Run headless mode so browser does not continually open with every iteration
options.add_argument("headless")
# Instatiate chrome driver and pass in the file path to chromedriver
driver = webdriver.Chrome('/Users/kevinmacmat/Desktop/capstone/chromedriver', options=options)

In [7]:
output_list = []

Get comments and output them to output_list

In [8]:
for dbn in dbn_list:
    # Get website 
    driver.get('https://insideschools.org/school/' + dbn)
    # Switch to iframe containing script tag
    driver.switch_to.frame(1)
    text = driver.page_source
    driver.switch_to.default_content()
    # Parse and process the source with BeautifulSoup module by creating an BS object
    soup = BeautifulSoup(text, 'lxml')
    # Access the soup and find the script element's id
    thread = soup.find("script", {"id": "disqus-threadData"})
#     thread = str(thread)
    # Turn the bs4 tag into a string, remove the script tag, and access the json
#     rm_script = thread.replace('<script id="disqus-threadData"type="text/json">', '')
#     rm_script = thread.replace('</script>', '')
    site_json=json.loads(str(thread)[48:-9])
#     site_json=json.loads(rm_script)
    # Instantiate comments list
    comments_list = []
    # Navigate and loop json, filtering comments by date, to append comments to comments_list
    for comment in site_json['response']['posts']:
        if '2014' in comment['createdAt']:
            comments_list.append(comment['raw_message'])
        elif '2015' in comment['createdAt']:
            comments_list.append(comment['raw_message'])
        elif '2016' in comment['createdAt']:
            comments_list.append(comment['raw_message'])
        elif '2017' in comment['createdAt']:
            comments_list.append(comment['raw_message'])
        elif '2018' in comment['createdAt']:
            comments_list.append(comment['raw_message'])
        elif '2019' in comment['createdAt']:
            comments_list.append(comment['raw_message'])
        elif '2020' in comment['createdAt']:
            comments_list.append(comment['raw_message'])
        else:
            continue
    # Append list of comments to output_list
    output_list.append(comments_list)

In [9]:
len(output_list)

98

In [10]:
# Export dataframe
batch_df = pd.DataFrame(dbn_list, columns=['dbn'])

In [11]:
# Add comments column to data frame
batch_df['comments'] = output_list

In [12]:
batch_df

,dbn,comments
0,30Q149,"[Hi, I am a former student that graduated in 2..."
1,30Q150,[PS 150 President Carmen Parache has created a...
2,30Q151,"[151 has come a long way, but don't let it foo..."
3,30Q152,"[Ps 152 queens, Horrible school! Rude staff, r..."
4,30Q166,[Schools Chancellor Carmen Fariña announced th...
...,...,...
93,32K377,"[Does anyone remember Mr. Corlito, 6th grade t..."
94,32K383,[The District 32 Community Education Council h...
95,32K384,[P.S./I.S. 384 principal Phyllis Raulli said c...
96,32K554,[for the 7th graders are you required to wear ...


Export completed data frame

In [13]:
batch_df.to_csv('batch_1000-end_comments.csv', index=False)